## **The Battle of the Neighborhoods - Week 2**

### **Explore New York city geographical coordinates dataset**

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

First, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

The "*json*" file is downloaded and it is placed on the server. So run a "*wget*" command and access the data.

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


**Load and explore the data**

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

All the relevant data is in the features key, which is basically a list of the neighborhoods. So, define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Take a look at the first item in this list

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

**Transform the data into a Pandas Dataframe**

The next task is essentially transforming this data of nested Python dictionaries into a Pandas Dataframe. Start by creating an empty Dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
neighborhoods.to_csv('BON1_NYC_GEO.csv',index=False)

**Use geopy library to get the latitude and longitude values of New York City.**

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Create a map of New York with neighborhoods superimposed on top.

'*Folium*' is a great visualization library. We can zoom into the below map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

In [12]:
# create map of NewYork using latitude and longitude values
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork)  
    
map_NewYork

### **Web scrapping of Population and Demographics data of New York city from Wikipedia**

#### **A : POPULATION DATA**

Web scrapping of Population data from wikipedia page 
https://en.wikipedia.org/wiki/New_York_City

Download all the dependencies that is needed.

In [13]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

     |████████████████████████████████| 102kB 8.8MB/s ta 0:00:011
     |████████████████████████████████| 92kB 14.4MB/s eta 0:00:01
     |████████████████████████████████| 829kB 6.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Packages installed.


In [14]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 22.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# conda install -c anaconda beautiful-soup --yes
from bs4 import BeautifulSoup # package for parsing HTML and XML documents

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Web scrapping of Population Data from wikipedia page using BeautifulSoup.

In [16]:
URL = 'https://en.wikipedia.org/wiki/Demographics_of_New_York_City'
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [17]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = BeautifulSoup(r.content, 'html5lib')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('BON2_POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)


**Load data from CSV**

In [18]:
Pop_data=pd.read_csv('BON2_POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[3,7,8,9,10,11,12,13,14]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


Remove white spaces and rename columns

In [19]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,Landarea\n,Density\n,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN


In [20]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n","30,100\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n","35,800\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n","368,500\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n","41,400\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n","30,500\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN


Replace newline('\n') from each string from left and right sides and delete unnecessary columns 

In [21]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2017']=Pop_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_miles']=Pop_data['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_km']=Pop_data['square_km'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)

Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,City of New York,"8,336,817",842.343,302.64,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","47,214","122,284",412
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN


In [22]:
Pop_data.iloc[5] = Pop_data.iloc[5].shift(1,axis=0)
Pop_data.iloc[6] = Pop_data.iloc[6].shift(1,axis=0)
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,NaN,City of New York,"8,336,817",842.343,302.64,783.83
6,NaN,State of New York,"19,453,561","1,731.910","47,214","122,284"
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN


Now let's remove NaN 

In [23]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,214","122,284"
7,Sources:[14] and see individual borough articles,,,,,


and the last unnecessary row 

In [24]:
i = Pop_data[((Pop_data.Borough == 'Sources:[14] and see individual borough articles'))].index
Pop_data.drop(i)

,Borough,County,Estimate_2017,square_miles,square_km,persons_sq_mi
0,The Bronx,Bronx,"1,418,207","30,100",42.10,109.04
1,Brooklyn,Kings,"2,559,903","35,800",70.82,183.42
2,Manhattan,New York,"1,628,706","368,500",22.83,59.13
3,Queens,Queens,"2,253,858","41,400",108.53,281.09
4,Staten Island,Richmond,"476,143","30,500",58.37,151.18
5,,City of New York,"8,336,817",842.343,302.64,783.83
6,,State of New York,"19,453,561","1,731.910","47,214","122,284"


Save Dataframe as CSV 

In [25]:
Pop_data.to_csv('BON2_POPULATION.csv',index=False)

### **B : DEMOGRAPHICS DATA**

We will web scrap Demographics data from wikipedia page 
- https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City

Web scrapping of Demographics data from wikipedia page using BeautifulSoup.

In [26]:
URL = 'https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City'
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [27]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City').text
soup = BeautifulSoup(website_url,'html5lib')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

Load data from CSV

In [28]:
Demo_data=pd.read_csv('NYC_DEMO.csv')
print('Data downloaded!')

Data downloaded!


In [29]:
Demo_data

,Year,Population,White(includes White Hispanics),%W,Non-Hispanic Whites,%ANG,Black,%B,Asian,%A,Other orMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB\n
0,1900,"3,437,202","3,369,898",98.04,NaN,NaN,"60,666",1.76,"6,607",0.19,31,0.00,NaN,NaN,"1,270,080",36.95
1,1910,"4,766,883","4,669,162",97.95,NaN,NaN,"91,709",1.92,"5,669",0.12,343,0.01,NaN,NaN,"1,944,357",40.79
2,1920,"5,620,048","5,459,463",97.14,NaN,NaN,"152,467",2.71,"7,969",0.14,149,0.00,NaN,NaN,"2,028,160",36.09
3,1930,"6,930,446","6,589,377",95.08,NaN,NaN,"327,706",4.73,"12,972",0.19,391,0.01,NaN,NaN,"2,358,686",34.03
4,1940,"7,454,995","6,977,501",93.59,"6,856,586",91.97,"458,444",6.15,"17,986",0.24,"1,064",0.01,"120,915",1.62,"2,138,657",28.69
5,1950,"7,891,957","7,116,441",90.17,NaN,NaN,"747,608",9.47,"21,441",0.27,"6,467",0.08,NaN,NaN,"1,784,206",22.61
6,1960,"7,781,984","6,640,662",85.33,NaN,NaN,"1,087,931",13.98,"43,103",0.55,"10,288",0.13,NaN,NaN,"1,558,690",20.03
7,1970,"7,894,862","6,048,841",76.62,"4,969,749",62.95,"1,668,115",21.13,"94,499",1.20,"83,407",1.06,"1,278,630",16.20,"1,437,058",18.20
8,1980,"7,071,639","4,294,075",60.72,"3,668,945",51.88,"1,784,337",25.23,"231,501",3.27,"761,762",10.77,"1,406,024",19.88,"1,670,199",23.62
9,1990,"7,322,564","3,827,088",52.26,"3,163,125",43.20,"2,102,512",28.71,"512,719",7.00,"880,245",12.02,"1,783,511",24.36,"2,082,931",28.45


In [30]:
Demo_data.columns

Index(['Year', 'Population', 'White(includes White Hispanics)', '%W',
       'Non-Hispanic Whites', '%ANG', 'Black', '%B', 'Asian', '%A',
       'Other orMixed', '%O/M', 'Hispanic/Latino', '%H/L', 'Foreignborn',
       '%FB\n'],
      dtype='object')

Remove NaN values 

In [31]:
Demo_data.columns = Demo_data.columns.str.replace(' ', '')

In [32]:
Demo_data= Demo_data.fillna('')
Demo_data


,Year,Population,White(includesWhiteHispanics),%W,Non-HispanicWhites,%ANG,Black,%B,Asian,%A,OtherorMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB\n
0,1900,"3,437,202","3,369,898",98.04,,,"60,666",1.76,"6,607",0.19,31,0.00,,,"1,270,080",36.95
1,1910,"4,766,883","4,669,162",97.95,,,"91,709",1.92,"5,669",0.12,343,0.01,,,"1,944,357",40.79
2,1920,"5,620,048","5,459,463",97.14,,,"152,467",2.71,"7,969",0.14,149,0.00,,,"2,028,160",36.09
3,1930,"6,930,446","6,589,377",95.08,,,"327,706",4.73,"12,972",0.19,391,0.01,,,"2,358,686",34.03
4,1940,"7,454,995","6,977,501",93.59,"6,856,586",91.97,"458,444",6.15,"17,986",0.24,"1,064",0.01,"120,915",1.62,"2,138,657",28.69
5,1950,"7,891,957","7,116,441",90.17,,,"747,608",9.47,"21,441",0.27,"6,467",0.08,,,"1,784,206",22.61
6,1960,"7,781,984","6,640,662",85.33,,,"1,087,931",13.98,"43,103",0.55,"10,288",0.13,,,"1,558,690",20.03
7,1970,"7,894,862","6,048,841",76.62,"4,969,749",62.95,"1,668,115",21.13,"94,499",1.20,"83,407",1.06,"1,278,630",16.2,"1,437,058",18.20
8,1980,"7,071,639","4,294,075",60.72,"3,668,945",51.88,"1,784,337",25.23,"231,501",3.27,"761,762",10.77,"1,406,024",19.88,"1,670,199",23.62
9,1990,"7,322,564","3,827,088",52.26,"3,163,125",43.2,"2,102,512",28.71,"512,719",7.00,"880,245",12.02,"1,783,511",24.36,"2,082,931",28.45


Save data as BON2_DEMOGRAPHICS.csv

In [33]:
Demo_data.to_csv('BON2_DEMOGRAPHICS.csv',index=False)

## **Download and Explore New York city and its Boroughs Cuisine dataset**

First we need to use Wikipedia API and Wordcloud.

In [34]:
#Install Wikipedia 
!pip install wikipedia

# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - wordcloud==1.4.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    olefile-0.46               |             py_0          31 KB  conda-forge
    pillow-7.1.2               |   py36h8328e55_0         656 KB  conda-forge
    wordcloud-1.4.1            |           py36_0         324 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        1011 KB

The following NEW packages will be INSTALLED:

  olefile            conda-forge/noarch::olefile-0.46-p

This data is extracted from the wikipedia Page 
- https://en.wikipedia.org/wiki/Cuisine_of_New_York_City

let's search for the page 

In [35]:
import wikipedia
print(wikipedia.search("Cuisine of New York City"))

['Cuisine of New York City', 'New York City', 'New York-style pizza', 'American cuisine', 'Jewish American Chinese restaurant patronage', 'Cannoli', 'Sausage and peppers', 'Italian sandwich', 'Bagel and cream cheese', 'Cuisine of New Jersey']


Let's print the page content of 'Cuisine of New York City'

In [36]:
NYC_CUISINE = wikipedia.page("Cuisine of New York City")
print(NYC_CUISINE.content)

The cuisine of New York City comprises many cuisines belonging to various ethnic groups that have entered the United States through the city. Almost all ethnic cuisines are well represented in New York City, both within and outside the various ethnic neighborhoods. New York City was also the founding city of New York Restaurant Week which has spread around the world due to the discounted prices that such a deal offers. In New York City there are over 12,000 bodegas, delis and groceries and many among them are open 24 hours a day, 7 days a week.


== Enclaves reflecting national cuisines ==


=== The Bronx ===


=== Queens ===


=== Brooklyn ===


=== Staten Island ===
Port Richmond – Mexican, Indian, Italian
Rossville; South Beach; Great Kills – Italian, Russian, Arab and Polish
Tompkinsville – Italian, Sri Lankan, Pakistani, Indian


=== Manhattan ===


== Food identified with New York City ==


=== Food associated with or popularized in New York City ===

Hot dogs – Served with sauer

In [37]:
stopwords = set(STOPWORDS)